In [1]:
!pip install elasticsearch

     |████████████████████████████████| 387 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 20.9 MB/s eta 0:00:01


Sample some examples from the Social IQa training set.

In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch(
         cloud_id="lm-datasets:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDk1N2U5ODIwZDUxNTQ0YWViMjk0MmQwNzI1NjE0OTQ2JDhkN2M0OWMyZDEzMTRiNmM4NDNhNGEwN2U4NDE5NjRl",
         api_key="T2w5Vl9ZTUIzVzE5dTlBblUtRlo6MHNBYWxhbHVTeGFodUpUOWIybkNNZw==",
         retry_on_timeout=True,
         http_compress=True)

In [13]:
import json
import random

with open("socialIQa_v1.4_trn.jsonl") as f_in:
    socialiqa_examples = [json.loads(line) for line in f_in]
    
sample = random.sample(socialiqa_examples, 100)

In [14]:
contexts = [ex["context"].split(".")[0] for ex in sample]
contexts = [c for c in contexts if len(c.split()) > 3]

Look for it in different corpora.

In [18]:
indices = [name for name in es.indices.get(index="*").keys() if not name.startswith(".")]

for corpus in indices:
    found = 0 
    for context in contexts:
        found += min(1, es.search(
            index=corpus, body={"query": {"match_phrase": {"text": context}}})['hits']['total']['value'])
    
    print(f"{corpus}: {found}%")

/var/folders/hz/7bs42t6d18s3r5qpwk86kvlw0000gn/T/ipykernel_60502/452525507.py:8: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  found += min(1, es.search(


c4: 0%
metrics-endpoint.metadata_current_default: 0%
openwebtext: 0%
re_laion2b-en-1: 0%
re_laion2b-en-2: 0%
re_oscar: 0%
re_pile: 0%
s2orc-abstracts: 0%
search-test: 0%
test-index: 0%
test-index-2: 0%


Sanity check with phrases that probably exist on the web:

In [20]:
fake_contexts = ["hello world", "last night I dreamt", "he works at"]

for corpus in indices:
    found = 0 
    for context in fake_contexts:
        found += min(1, es.search(
            index=corpus, body={"query": {"match_phrase": {"text": context}}})['hits']['total']['value'])
    
    print(f"{corpus}: {found}/{len(fake_contexts)}")

/var/folders/hz/7bs42t6d18s3r5qpwk86kvlw0000gn/T/ipykernel_60502/40817552.py:6: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  found += min(1, es.search(


c4: 3/3
metrics-endpoint.metadata_current_default: 0/3
openwebtext: 3/3
re_laion2b-en-1: 3/3
re_laion2b-en-2: 3/3
re_oscar: 3/3
re_pile: 3/3
s2orc-abstracts: 2/3
search-test: 0/3
test-index: 1/3
test-index-2: 1/3
